In [19]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [49]:
path_data_dir = "/home/juan/Desktop/juan/bio/mirna_mite/data/res/mrcv_all/"
path_res_dir = "/home/juan/Desktop/juan/bio/mirna_mrcv/data/"
path_fasta_res = "/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.deg.Y.fasta"
path_mirbase_res = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.y.mature.csv'
path_deg_file = path_res_dir + "mirna.deg.csv"
path_res_ann =  path_res_dir + "mirna.res.deg.ann.csv"

In [30]:
path_results = path_data_dir + "Results.txt"
df_res = pd.read_csv(path_results, sep="\t")
df_res.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1A:209-460,Cluster_1,252,118,0.926,64,1.0,+,CGCGGAUUGCUCGAGCUGCUCACGCGGCGAGAGCGGGUCG,25,...,N,N2,-1.0,28,71,2,1,6,6,4
1,1A:690-750,Cluster_2,61,78,0.612,0,1.0,+,UAGACCGACUUUGUGAAAUG,5,...,N,N2,-1.0,7,54,7,0,1,2,7


In [31]:
df_deg = pd.read_csv(path_deg_file, sep=",")
print(len(df_deg.index))
df_deg.head(2)

11


,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed
0,Cluster_9889,298.598043,-1.471187,0.264059,-5.571425,2.526645e-08,0.000002,417.884204,459.336142,137.863421,179.308403
1,Cluster_35781,411.843895,-0.908261,0.236510,-3.840258,1.229050e-04,0.005899,583.212178,491.732645,306.089619,266.341137


In [32]:
len(df_deg[df_deg.log2FoldChange > 0].index), len(df_deg[df_deg.log2FoldChange < 0].index)

(4, 7)

In [33]:
df_deg = df_deg[['Name','log2FoldChange']]

In [34]:
df_deg_y = pd.merge(df_res, df_deg, on='Name')
print(len(df_deg_y.index))
df_deg_y.head(2)

11


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange
0,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2883,...,Y,-1.0,1080,3,4050,798,52,0,2,-0.915816
1,2A:652328872-652329011,Cluster_6309,140,1185,9.300,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,Y,-1.0,14,0,3,1155,11,2,0,-1.201318


In [35]:
buffer_seqs = []
for k,v in df_deg_y.iterrows():
    description = '%s l2fc:%f' % (v['#Locus'], v['log2FoldChange'])
    seq = SeqRecord(Seq(v.MajorRNA), id=v.Name, description=description)
    buffer_seqs.append(seq)
SeqIO.write(buffer_seqs, path_fasta_res, "fasta")
print(len(buffer_seqs))
print(path_fasta_res)

11
/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.deg.Y.fasta


blastn -task blastn -evalue 10 -query data/mirna.deg.Y.fasta -subject data/mature.fa -outfmt '6 qseqid sseqid qstart qend sstart send mismatch gaps pident evalue length qlen slen qcovs score'  > data/mirna.y.mature.csv


In [37]:
df_mirbase = pd.read_csv(path_mirbase_res, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs','score']
df_mirbase.columns = cols
print(len(df_mirbase.index))
df_mirbase.head(2)


447


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
0,Cluster_5906,sbi-miR169d-3p,3,20,2,19,0,0,100.000,0.00021,18,20,20,90,36
1,Cluster_5906,bdi-miR169h-3p,3,20,1,18,1,0,94.444,0.00900,18,20,19,90,30


In [38]:
allowed_mirbase_mismatches = 2
mirbase_qlen = 2

In [39]:
df_mirbase = df_mirbase[df_mirbase.length > df_mirbase.qlen - mirbase_qlen]
print(len(df_mirbase.index))


318


In [40]:
df_mirbase = df_mirbase[df_mirbase.mismatch + df_mirbase.gaps <= allowed_mirbase_mismatches]
print(len(df_mirbase.index))


315


In [41]:
len(df_mirbase.qseqid.unique().tolist())


9

In [42]:
df_mirbase = df_mirbase.loc[df_mirbase.groupby("qseqid")["score"].idxmax()]

In [43]:
print(len(df_mirbase.index))
df_mirbase.head(2)


9


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
228,Cluster_10582,tae-miR9776,1,20,2,21,1,0,95.0,0.000650,20,21,21,95,34
232,Cluster_17243,bdi-miR169d,1,20,1,20,0,0,100.0,0.000017,20,20,21,100,40


In [44]:
df_mirbase = df_mirbase[['qseqid','sseqid']]
df_mirbase.rename(columns={'sseqid':'mirbase'}, inplace=True)

In [45]:
print(len(df_mirbase.index))
df_mirbase.head(2)


9


,qseqid,mirbase
228,Cluster_10582,tae-miR9776
232,Cluster_17243,bdi-miR169d


In [46]:
df_deg_y.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange
0,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2883,...,Y,-1.0,1080,3,4050,798,52,0,2,-0.915816
1,2A:652328872-652329011,Cluster_6309,140,1185,9.300,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,Y,-1.0,14,0,3,1155,11,2,0,-1.201318


In [47]:
df_1 = pd.merge(df_deg_y, df_mirbase, how='left', left_on='Name', right_on='qseqid')
df_1.drop(['qseqid'], axis=1, inplace=True)
df_1.head(2)
print(len(df_1.index))

11


In [51]:
df_1.to_csv(path_res_ann, sep="\t", index=None)
path_res_ann

'/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.res.deg.ann.csv'